<a href="https://colab.research.google.com/github/jinseriouspark/commerce-llm/blob/main/extract_keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# extract- keywords
- 진행일자 : 24.03.26
- 목적 : 해당 리뷰를 분리할 수 있는 핵심 키워드를 추출
- 비교항목 :
- 판단 기준 : 1~5를 가장 잘 분리할 수 있는 모델


In [ ]:
with open('/content/drive/MyDrive/commerce-llm/rawdata/naver_shopping.txt', 'r', encoding = 'utf-8') as f:
  data =f.readlines()

In [ ]:
rawdata =[i.rstrip().split('\t') for i in data]

In [ ]:
len(rawdata)

200000

- 20만개를 가지고 하기엔 1개 모델당 4시간 정도가 걸림
- 판단기준을 주어진 시간 내 잘 검증할 수 있도록 샘플 데이터를 아래와 같이 추출

In [ ]:
import pandas as pd
raw_df = pd.DataFrame(rawdata)

In [ ]:
raw_df.head()

,0,1
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [ ]:
features = raw_df[1]
target = raw_df[0]

In [ ]:
raw_df[0].value_counts() / raw_df.shape[0]
# 3번은 존재하지 않음
# 1 18%, 2 32%, 4 9.3%, 5 40%

5    0.405885
2    0.319945
1    0.180240
4    0.093930
Name: 0, dtype: float64

In [ ]:
min_cnt = raw_df[0].value_counts().min()
raw_df_for_test = raw_df.groupby(0).apply(lambda x : x.sample(min_cnt)).reset_index(drop = True)

raw_df_for_test.shape[0] / raw_df.shape[0] # 37% 만 가지고 테스트셋으로 만들어 진행?

0.37572

In [ ]:
# baseline : 만약 이 모든 값들을 그냥 분류한다면 어떻게 될까?
model_name = "monologg/kobert"
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# prompt: make torch Dataset  with korean text, and labels

import torch

class KoreanTextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded_text = self.tokenizer(self.texts[idx], return_tensors='pt')
        label = torch.tensor(self.labels[idx])
        return encoded_text, label


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class myDataset(Dataset):
  def __init__(self, text, label, tokenizer):
    self.text = text
    self.label = label
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    encoded_text = [self.tokenizer(self.text[i], return_tensors = 'pt') for i in idx]
    label = torch.tensor(self.label[idx])
    return encoded_text, label

In [ ]:
raw_df_for_test.columns = ['target','review']

In [ ]:
raw_df_for_test.to_json('/content/drive/MyDrive/commerce-llm/rawdata/rawdata', orient = 'records', lines = True)



In [ ]:
!pip install datasets
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
load_dataset('/content/drive/MyDrive/commerce-llm/rawdata/rawdata')

array(['너무 끈적거려요...풀 같아요...... 로션바르고 다시 씻겼어요........',
       '제가 주문을 잘못한거 같아서 주문내역보니 제대로 98파이로 주문한거 같은데요. 뚜껑은 92파이로 왔네요. 진짜 뚜껑 열리네요.',
       '고장인것같은데 물건부붐이 재대로 안와서 장롱에 처박아 놔ㅛ어요', ...,
       '가성비 좋네요^^ 생각보다 베게도 베게솜도 굿이네여👍',
       '보테가베네타 지갑 처음써보는데 좋네요 매장a/s시 필요한 카드도 주시고 정품 확실한것같아요 배송도 빠르 맘에듭니다 많이 파세요^^',
       '미싱옆에 실 정리가 깔끔하게 되었어요 배송도 빠르고 만족합니다'], dtype=object)

In [ ]:
ds = myDataset(raw_df_for_test[1].values, raw_df_for_test[0].values, tokenizer)

In [ ]:
ds[:3]

TypeError: 'slice' object is not iterable

# 비교 모델
- [참고자료](https://https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/)

1. Rake_NLTK
2. Spacy
3. Textrank
4. Word Cloud
5. KeyBert
6. Yet Another Keyword Extractor Yake
7. MonkeyLearn API
8. Textrazor API
9. applied NLI

In [ ]:
# KEYWORD EXTRACTION BY keybert

# KEYWORD EXTRACTION BY keybert
- keybert 란?

In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 751.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 k

In [ ]:
from keybert import KeyBERT
from tqdm import tqdm

model = KeyBERT(model="all-MiniLM-L6-v2")
keywords_list = []
for raw in tqdm(rawdata[:100]):
  keywords = model.extract_keywords(raw)
  keywords_list.append(keywords)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 100/100 [00:05<00:00, 17.55it/s]


In [ ]:
len(keywords_list)

578

In [ ]:
import pandas as pd
result_df = pd.DataFrame(data = {'origin' : rawdata[:len(keywords_list)], 'keywords': keywords_list})

In [ ]:
result_df.head()

,origin,keywords
0,"[5, 배공빠르고 굿]","[[], [(배공빠르고, 0.9021)]]"
1,"[2, 택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고]","[[], [(엉망이네용, 0.7305), (저희집, 0.6034), (놔두고가고, ..."
2,"[5, 아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이...","[[], [(엉성하긴, 0.7211), (대박입니다, 0.6205), (가성비, 0..."
3,"[2, 선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니...","[[], [(사는거, 0.7021), (머그컵만, 0.6322), (확인안하고, 0..."
4,"[5, 민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ]","[[], [(민트색상, 0.7763), (손잡이는, 0.7663), (용도로도, 0..."


In [ ]:
# extrack keywords with mistral 7b
# https://iamshobhitagarwal.medium.com/running-mistral-7b-large-language-model-on-google-colab-gpu-completely-free-of-cost-fc0a57279de9

In [ ]:
!git lfs install
!git clone https://huggingface.co/mistralai/Mistral-7B-v0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0
!pip install -q peft==0.4.0
!pip install -qbitsandbytes==0.40.2
!pip install -q transformers==4.31.0
!pip install -q trl==0.4.7

     ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/23.7 MB 18.7 MB/s eta 0:00:02ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/23.7 MB 21.1 MB/s eta 0:00:01
     ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/23.7 MB 19.9 MB/s eta 0:00:01ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

     ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/23.7 MB 19.9 MB/s eta 0:00:01
     ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/23.7 MB 18.2 MB/s eta 0:00:01
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 527, in 

In [ ]:
!nvidia-smi

Wed Mar 27 12:44:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [ ]:
model_path= '/content/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             trust_remote_code = True,
                                             device_map = 'auto',
                                             load_in_8bit = True,
                                             max_memory = {0:"15GB"})


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
pip install accelerate

  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/23.7 MB 16.0 MB/s eta 0:00:02
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [ ]:
model.eval()

NameError: name 'model' is not defined